In [37]:
import numpy as np
import networkx as nx
import matplotlib.pylab as plt
import pandas as pd



%matplotlib inline


#assuming the importation of data as pandas data frames before here

#====begin making graph/network====


dfP=pd.read_csv('./data/final.csv')
dfP.rename(columns={'Category':'crimeCount'}, inplace=True)

## set true to draw all of the plots
drawPlots = False;




/Users/bradygerndt/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
#Making networks
from networkx.algorithms import bipartite
teamsGraph=nx.Graph()
teams=dfP.team.unique()


for i in teams:
	teamsGraph.add_node(i,team=1,bipartite=0,crimes=-1) #team name

names=dfP.name.unique()	



for name in names:#number of players
  
    nameTemp=dfP[(dfP.name==name)]

    playerCrimes=nameTemp.crimeCount.unique()
  
    crimee=0
    for crim in playerCrimes:
        if crim==1:
            crimee=1
    teamsGraph.add_node(name,crimes=crimee, bipartite=1)
    
    playerTeams=nameTemp.team.unique()
    for team in playerTeams:
        teamsGraph.add_edge(team,name)

		
    


In [39]:
#Actual network analysis


interestNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['bipartite']==0 or m['crimes']==1)]
crimeNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['crimes']==1)]
notCrimeNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['crimes']==0)]

teamNodes=[n for n,m in teamsGraph.nodes(data=True) if (m['crimes']==-1)]
listOfEdges = [edge for edge in teamsGraph.edges()]

## see https://stackoverflow.com/questions/16910538/efficient-extraction-of-a-subgraph-according-to-some-edge-attribute-in-networkx
crimeNet=teamsGraph.subgraph(interestNodes)
crimeEdges= [edge for edge in crimeNet.edges()]
#
#
##cool network analysis 
C=nx.clustering(crimeNet)
B=nx.betweenness_centrality(crimeNet)
crAdjMat=crimeNetAdjMat = nx.adjacency_matrix(crimeNet)
#
#ax(B[ x)
import operator
maxBet=max(B.items(), key=operator.itemgetter(1))[0]
minBet=min(B.items(), key=operator.itemgetter(1))[0]





In [40]:
if drawPlots:
    pos = nx.spring_layout(crimeNet)
    nx.draw_networkx_nodes(crimeNet, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=10)
    nx.draw_networkx_nodes(crimeNet, pos, nodelist=teamNodes, cmap=plt.get_cmap('jet'),node_color = 'g',node_size=10)

    nx.draw_networkx_edges(crimeNet, pos, edgelist=crimeEdges, arrows=False)

In [41]:
if drawPlots:  
    pos = nx.spring_layout(teamsGraph)
    nx.draw_networkx_nodes(teamsGraph, pos, nodelist=notCrimeNodes, cmap=plt.get_cmap('jet'), node_color = 'b',node_size=10)
    nx.draw_networkx_nodes(teamsGraph, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=10)
    nx.draw_networkx_nodes(teamsGraph, pos, nodelist=teamNodes, cmap=plt.get_cmap('jet'),node_color = 'g',node_size=10)
    #nx.draw_networkx_labels(teamsGraph, pos)
    nx.draw_networkx_edges(teamsGraph, pos, edgelist=listOfEdges, arrows=False)
    #plt.figure(figsize=)
    #plt.savefig("fullNet.png", dpi=1000)
    #plt.savefig("fullNet.pdf")
    plt.show()


In [42]:
playerNodes = (notCrimeNodes + crimeNodes)

uniTeamsGraph = bipartite.projected_graph(teamsGraph, playerNodes)

uniEdges = uniTeamsGraph.edges()


In [43]:
if drawPlots:
    pos = nx.spring_layout(uniTeamsGraph)
    nx.draw_networkx_nodes(uniTeamsGraph, pos, nodelist=notCrimeNodes, cmap=plt.get_cmap('jet'), node_color = 'b',node_size=10)
    nx.draw_networkx_nodes(uniTeamsGraph, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=10)
    nx.draw_networkx_edges(uniTeamsGraph, pos, edgelist=uniEdges, arrows=False)


In [44]:
uniCrimesGraph = bipartite.projected_graph(crimeNet, crimeNodes)
uniCrimeEdges = uniCrimesGraph.edges()

if drawPlots:
    nx.draw(uniCrimesGraph)


In [45]:
crimeGraph=nx.Graph()
crimeList = ['Alcohol','Animal Abuse','Assault / Battery', 'DUI','Disorderly conduct','Domestic Violence','Drugs','Gun','License / Traffic','Murder / Manslaughter','Resisting','Sex','Other','Theft / Burglary']

for i in crimeList:
    crimeGraph.add_node(i,issaCrime=1)
    
dfC = dfP[(dfP.crimeCount >= 1)]

for i in dfC.name.unique():
    crimeGraph.add_node(i, issaCrime=0)


def createCrimeNodes(i, row):
    for crime in crimeList:
        if row[crime] >= 1:
            crimeGraph.add_edge(crime, row['name'])
[createCrimeNodes(i, row) for i, row in dfC.iterrows()]

# print(dict([n for n, m in crimeGraph.nodes(data=True) if (m['issaCrime']==1)]))
labels = dict((key, key) for (key, value) in crimeGraph.nodes(data=True) if (value['issaCrime']==1))

crimeNodes =[n for n,m in crimeGraph.nodes(data=True) if (m['issaCrime']==1)]
playNodes =[n for n,m in crimeGraph.nodes(data=True) if (m['issaCrime']==0)]
listOfCrimeEdges= [edge for edge in crimeGraph.edges()]


In [46]:
#printing graph that links players crimes to the crimes
if drawPlots:
    pos = nx.spring_layout(crimeGraph)
    nx.draw_networkx_nodes(crimeGraph, pos, nodelist=crimeNodes, cmap=plt.get_cmap('jet'), node_color = 'r',node_size=20)
    nx.draw_networkx_nodes(crimeGraph, pos, nodelist=playNodes, cmap=plt.get_cmap('jet'),node_color = 'g',node_size=10)
    nx.draw_networkx_edges(crimeGraph, pos, edgelist=listOfCrimeEdges, arrows=False)
    nx.draw_networkx_labels(crimeGraph, pos, labels=labels)



In [47]:
###Degree centrality by team in crime net
D = crimeNet.degree()
degrees = sorted(set(D.values()))
degree_agg = [list(D.values()).count(x) for x in degrees]
    
teamDegrees = {}
for key in list((D))[:32]:
    teamDegrees[key] = D[key]

##identify team nodes for centrality measures
teamNodes = set(n for n,d in crimeNet.nodes(data=True) if d['bipartite']==0)

##Closeness by team in crime net
TC = bipartite.closeness_centrality(crimeNet, teamNodes)

teamCloseness = {}
for key in list((TC))[:32]:
    teamCloseness[key] = TC[key]

##Betweenness by team in crime net
TB = bipartite.betweenness_centrality(crimeNet, teamNodes)

teamBetweenness = {}
for key in list((TB))[:32]:

    teamBetweenness[key] = TB[key]


if drawPlots:
    plt.figure(figsize=(10,8))
    plt.plot(degrees, degree_agg)

    plt.legend('')
    plt.ylabel('Number of nodes (Player/Team)')
    plt.xlabel('Degree')
    plt.ylim(1,200)

    plt.title("Degree metrics for Crime Network")
    plt.savefig("DegreeMet.png")

if drawPlots:
    plt.figure(figsize=(10,8))
    plt.bar(range(len(teamBetweenness)), teamBetweenness.values())
    plt.xticks(range(len(teamBetweenness)), teamBetweenness.keys(), rotation=90)
    plt.title("Betweenness Centrality for NFL teams in the Crime Network")
    plt.ylabel('Betweenness')
    plt.xlabel("Team")
    plt.savefig("BetweenMet.png")
    plt.show()


In [48]:
##Metrics to add to data


##takes too long for the wholegraph
##closeness in one-mode graph
#cl = nx.closeness_centrality(uniTeamsGraph)
#bt = nx.betweenness_centrality(uniTeamsGraph)


##we'll go with team crime graph metrics

# TC = bipartite.closeness_centrality(crimeNet, teamNodes)

# teamCloseness = {}
# for key in list((TC))[:32]:
#     teamCloseness[key] = TC[key]
    
# teamDegrees = {}
# for key in list((D))[:32]:
#     teamDegrees[key] = D[key]



def matchNetworkMeasures(team):
    dfP.loc[(dfP.team == team), 'Team_Degrees'] = teamDegrees[team]
    dfP.loc[(dfP.team == team), 'Team_Closeness'] = teamCloseness[team]
    dfP.loc[(dfP.team == team), 'Team_Betweenness'] = teamBetweenness[team]

[matchNetworkMeasures(team) for team in dfP.team.unique()]

#matchNetworkMeasures('NYJ')

#dfP.loc[(dfP.team == "NYJ"), ['Team_Degrees','Team_Closeness','Team_Betweenness']]

    
dfP.to_csv('./data/final_networked_by_team.csv', index=False)